ロジスティック回帰

In [134]:
import pandas as pd

pd.options.display.width = 500
pd.options.display.column_space = 200

In [135]:
results = pd.read_pickle("df.pickle")

In [136]:
# results

In [137]:
clip_rank = lambda x: x if x < 4 else 4 

results["rank"] = results["着順"].map(clip_rank)

In [138]:
results["rank"].value_counts()

4    1897
1     173
2     173
3     173
Name: rank, dtype: int64

In [139]:
results["騎手"].value_counts()

☆富田     46
木幡巧     46
丹内      45
鮫島駿     41
☆菅原明    37
        ..
井上       2
難波       2
柴田未      2
三津谷      1
城戸義      1
Name: 騎手, Length: 147, dtype: int64

In [140]:
pd.set_option('display.max_rows',10)
results.drop(["着順","馬名"],axis=1,inplace=True)
results_d = pd.get_dummies(results)
results_d

,枠,斤量,人気,齢,体重,体重変化,rank,馬番_1,馬番_2,馬番_3,...,騎手_高田,騎手_高野,騎手_鮫島良,騎手_鮫島駿,騎手_黒岩,騎手_黛,騎手_Ｍデム,性_セ,性_牝,性_牡
202101010101,5.0,52.0,7.0,3,474,-22,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
202101010101,3.0,56.0,2.0,3,470,-2,2,0,0,0,...,0,0,0,0,0,0,0,0,0,1
202101010101,7.0,54.0,14.0,3,394,2,3,0,0,0,...,0,0,0,0,0,0,0,0,1,0
202101010101,5.0,54.0,5.0,3,468,2,4,0,0,0,...,0,0,0,1,0,0,0,0,1,0
202101010101,3.0,54.0,4.0,3,456,8,4,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202110040302,3.0,56.0,6.0,3,500,-18,4,0,0,0,...,0,0,0,0,0,0,0,0,0,1
202110040302,3.0,54.0,4.0,3,414,0,4,0,0,0,...,0,0,0,0,0,0,0,0,1,0
202110040302,5.0,56.0,12.0,3,500,6,4,0,0,0,...,0,0,0,0,0,0,0,0,0,1
202110040302,7.0,56.0,10.0,3,490,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [141]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X >> 学習させるデータ , y >> 予測したいデータ

In [142]:
X = results_d.drop(["rank"],axis=1)
y = results_d["rank"] 
print(X.shape)
print(y.shape)

(2416, 185)
(2416,)


データを２分割にする: train >> 学習データ : test >> テストデータ

In [143]:
X_train,X_test,y_train,y_test =  train_test_split(X,y, stratify=y, test_size=0.3,random_state=0)
print(X_train.shape)
print(X_test.shape)

(1691, 185)
(725, 185)


In [144]:
print(X_train)
print(X_test)
print(y_train)
print(y_test)

                枠    斤量    人気  齢   体重  体重変化  馬番_1  馬番_2  馬番_3  馬番_4  ...  騎手_高田  騎手_高野  騎手_鮫島良  騎手_鮫島駿  騎手_黒岩  騎手_黛  騎手_Ｍデム  性_セ  性_牝  性_牡
202103010301  7.0  54.0   1.0  2  422     2     0     0     0     0  ...      0      0       0       0      0     0       0    0    1    0
202104020301  7.0  53.0   8.0  3  438     8     0     0     0     0  ...      0      0       0       0      0     0       0    0    1    0
202104040201  7.0  60.0   6.0  4  482    -6     0     0     0     0  ...      0      0       0       0      0     0       0    0    0    1
202107010302  5.0  56.0  13.0  3  460    -2     0     0     0     0  ...      0      0       0       0      0     0       0    0    0    1
202107030302  8.0  56.0   1.0  3  486     2     0     0     0     0  ...      0      0       0       0      0     0       0    0    0    1
...           ...   ...   ... ..  ...   ...   ...   ...   ...   ...  ...    ...    ...     ...     ...    ...   ...     ...  ...  ...  ...
202110020201  3.0  53.0   2

In [145]:
model = LogisticRegression()
model.fit(X_train,y_train)

/Users/masa/Desktop/scikit-learn-test/venv39/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [146]:
print(model.score(X_train,y_train),model.score(X_test,y_test))

0.7859254878769959 0.7875862068965517


In [147]:
y_pred = model.predict(X_test)
pd.Series(y_pred).value_counts()

4    719
2      5
1      1
dtype: int64

In [148]:
from imblearn.under_sampling import RandomUnderSampler

rank_1 = y_train.value_counts()[1]
rank_2 = y_train.value_counts()[2]
rank_3 = y_train.value_counts()[3]

rus = RandomUnderSampler(sampling_strategy="auto",random_state=None,replacement=False)
X_train_rus,y_train_rus = rus.fit_resample(X_train,y_train)

In [149]:
pd.Series(y_train_rus.value_counts())

1    121
2    121
3    121
4    121
Name: rank, dtype: int64

In [150]:
model = LogisticRegression()
model.fit(X_train_rus,y_train_rus)

/Users/masa/Desktop/scikit-learn-test/venv39/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [151]:
print(model.score(X_train,y_train),model.score(X_test,y_test))

0.6179775280898876 0.5958620689655172


In [152]:
y_pred = model.predict(X_test)
y_pred

array([1, 4, 4, 3, 3, 4, 4, 2, 1, 4, 4, 4, 4, 4, 3, 4, 4, 1, 2, 3, 1, 4,
       4, 2, 4, 4, 3, 1, 1, 3, 4, 2, 2, 4, 4, 4, 2, 3, 4, 4, 4, 1, 4, 1,
       1, 3, 4, 4, 2, 4, 2, 4, 4, 4, 1, 4, 4, 4, 1, 2, 3, 4, 4, 2, 2, 4,
       3, 1, 4, 2, 4, 1, 4, 4, 4, 2, 2, 4, 3, 4, 2, 2, 3, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 3, 2, 4, 4, 4, 1, 2, 2, 3, 4, 4, 4, 4, 4, 1, 1,
       3, 4, 3, 3, 4, 1, 2, 1, 2, 1, 1, 4, 4, 1, 1, 4, 4, 4, 4, 4, 3, 3,
       1, 3, 4, 1, 3, 4, 4, 3, 4, 3, 2, 4, 4, 4, 4, 1, 4, 2, 4, 3, 4, 4,
       4, 2, 4, 4, 2, 3, 2, 4, 4, 3, 4, 2, 1, 4, 4, 3, 4, 3, 1, 4, 4, 3,
       4, 4, 2, 4, 2, 4, 3, 4, 3, 3, 1, 1, 3, 4, 4, 2, 4, 3, 3, 2, 4, 4,
       4, 3, 2, 4, 2, 4, 4, 3, 4, 4, 3, 4, 4, 1, 2, 2, 4, 4, 4, 4, 4, 4,
       4, 4, 2, 2, 4, 3, 1, 3, 4, 4, 2, 2, 4, 4, 4, 2, 4, 1, 4, 4, 4, 2,
       2, 3, 4, 4, 4, 3, 3, 4, 4, 3, 4, 4, 3, 2, 3, 4, 4, 1, 4, 1, 4, 4,
       4, 3, 4, 4, 4, 2, 4, 2, 4, 2, 4, 4, 4, 4, 4, 4, 1, 4, 1, 2, 4, 4,
       4, 1, 4, 2, 4, 1, 3, 4, 2, 4, 4, 3, 4, 3, 4,

In [155]:
pred_df = pd.DataFrame({"pred":y_pred,"actual":y_test})
pred_df

,pred,actual
202105040301,1,1
202101010102,4,4
202104040101,4,4
202104010102,3,4
202107030202,3,1
...,...,...
202107020201,4,4
202109030201,2,2
202110010101,3,4
202110020101,1,4


In [157]:
pred_df[pred_df["pred"] == 1]["actual"].value_counts()

4    51
2    12
1     8
3     7
Name: actual, dtype: int64

In [158]:
len(pred_df[pred_df["pred"] == 1])

78

In [159]:
pred_df[pred_df["pred"] == 4]["actual"].value_counts()

4    391
2     12
3     11
1      6
Name: actual, dtype: int64

In [165]:
pd.set_option('display.max_rows',None)

coefs = pd.Series(model.coef_[0],index=X.columns).sort_values()
coefs

齢         -0.175560
人気        -0.093084
馬番_1      -0.078228
騎手_☆富田    -0.058404
馬番_6      -0.058320
騎手_三浦     -0.056414
馬番_11     -0.050863
騎手_和田竜    -0.045025
騎手_▲角田    -0.039314
騎手_岩田望    -0.036156
騎手_☆斎藤    -0.034271
騎手_▲永野    -0.031797
騎手_丸山     -0.029467
騎手_▲小沢    -0.027919
騎手_池添     -0.027893
騎手_☆菅原明   -0.026517
騎手_西谷誠    -0.024685
馬番_2      -0.021766
騎手_北村宏    -0.021242
騎手_菱田     -0.020317
騎手_吉田隼    -0.019753
馬番_8      -0.019052
騎手_高田     -0.018698
性_牡       -0.017959
騎手_野中     -0.017910
騎手_菅原明    -0.017047
騎手_幸      -0.016682
騎手_浜中     -0.016219
騎手_国分優    -0.013827
騎手_☆団野    -0.013810
騎手_☆木幡育   -0.013382
騎手_黒岩     -0.013151
騎手_酒井     -0.013078
騎手_小野寺    -0.012801
騎手_白浜     -0.012732
騎手_北沢     -0.012066
馬番_15     -0.012007
騎手_丹内     -0.011601
騎手_吉田豊    -0.011436
騎手_黛      -0.011117
騎手_森一     -0.010298
馬番_12     -0.009908
騎手_津村     -0.009708
騎手_古川吉    -0.009168
騎手_平沢     -0.009066
騎手_藤井勘    -0.008140
騎手_加藤     -0.008031
騎手_小牧     -0.007732
騎手_荻野極    -0.007479
馬番_10     -0.007350


In [169]:
results[results["騎手"] == "ルメー"]["rank"].value_counts()

1    9
3    6
2    6
4    6
Name: rank, dtype: int64